In [51]:
!pip install transformers
!pip install pyyaml h5py

In [52]:
import numpy as np
import matplotlib.pyplot as plt

import random
from random import shuffle

import itertools

import IPython.display as ipd

import time
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix
from keras.models import Model, Sequential, load_model
from keras.layers import Flatten, Dense, Lambda, Dropout
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16
from keras.optimizers import Adam
from keras.preprocessing import image
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import smart_resize
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.layers import InputLayer, BatchNormalization, Dropout, Flatten, Dense, Activation, MaxPool2D, Conv2D

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip '/content/drive/MyDrive/solder/trainval.zip'

Archive:  /content/drive/MyDrive/solder/trainval.zip
replace __MACOSX/._trainval? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
model = load_model('/content/drive/MyDrive/solder/model/vggbase2.h5')

In [ ]:
seed = 1337

random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

batch_size = 64
target_size = (224, 224)

labels = np.array(['excessive_solder', 'insufficient_solder', 'normal',
       'shifted_component', 'short'])

valtest_gen = image.ImageDataGenerator(rescale = 1./255,
                                       fill_mode='nearest',
                                       )


In [ ]:
def get_predictions(directory, filename):
    test_data = valtest_gen.flow_from_directory(directory,
                                target_size=(224,224),
                                class_mode='categorical',
                                shuffle=False,
                                batch_size=batch_size)
    f_names = np.array(test_data.filenames)
    predictions = np.array(model.predict(test_data))
    resnet_predictions = np.column_stack((f_names, predictions))
    resnet_predictions[:,0] = np.char.replace(resnet_predictions[:,0], "test/", "")
    resnet_predictions[:,0] = np.char.replace(resnet_predictions[:,0], ".jpg", "")
    categories = np.insert(labels, 0, 'image', axis=0)
    resnet_predictions = np.insert(resnet_predictions, 0, categories, 0)
    np.savetxt(filename, resnet_predictions, delimiter=",", fmt="%s")

In [ ]:
def get_predictions_one(directory):
    test_data = valtest_gen.flow(directory,
                                target_size=(224,224),
                                class_mode='categorical',
                                shuffle=False,
                                batch_size=batch_size)
    predictions = np.array(model.predict(test_data))
    predictions = np.argmax(predictions, axis = 1)
    return [labels[i] for i in predictions]

In [ ]:
# Predictions with all possibilities
get_predictions('trainval/val', 'resnet_valid_predictions.csv')

Found 511 images belonging to 5 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/preprocessing/image.py:202: UserWarning: Using ".tiff" files with multiple bands will cause distortion. Please verify your output.
  warnings.warn(


8/8 [==============================] - 110s 13s/step


In [ ]:
# Predictions with top 1
pre_label = get_predictions_one('trainval/val')

Found 511 images belonging to 5 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/preprocessing/image.py:202: UserWarning: Using ".tiff" files with multiple bands will cause distortion. Please verify your output.
  warnings.warn(


8/8 [==============================] - 109s 13s/step
